# 1.OpenAPI을 이용하여 데이터 수집

In [9]:
import requests
import pandas as pd
import json

#1. 타겟 사이트 url
url = "http://data4library.kr/api/loanItemSrch?format=json&startDt=2023-01-01&endDt=2023-12-30&age=20&authKey="
# 2. 페이지 로딩
res = requests.get(url)

if res.status_code == 200:
    print("Request successful")
else:
    print("Error:", res.status_code)
    
# print(res.text)

# # # Decode binary content to UTF-8 string
# # content_str = res.content.decode('utf-8')

# # 3. Parse the JSON data
data = json.loads(res.text)
# # data = json.loads(content_str)


# 4. Print the parsed JSON data
# print(data)

# 5. ETL
# # 5-1 데이터 추출
books = []
for d in data['response']['docs']:
    books.append(d['doc'])

# print(books[0])
# # books = [d['doc'] for d in data['response']['docs']] # list complrehension

# 5-2 리스트을 이용하여  데이터프레임 생성:  list -> DataFrame
books_df = pd.DataFrame(books)

books_df.head(2)

# 5-3 dataFrame 객체을 json 파일로 저장
books_df.to_json('20s_best_book_2023.json')



Request successful


## 2. Open API을 활용한 데이터셋 가공 작업 :  "20대가 가장 좋아하는 도서목록"

In [10]:
# 1. json file 로딩 -> dataFrame 객체 변환
books_df = pd.read_json('20s_best_book_2023.json')
books_df.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13,addition_symbol,vol,class_no,class_nm,loan_count,bookImageURL,bookDtlUrl
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156,03400,,490.99,자연과학 > 동물학 > 동물학,5907,https://image.aladin.co.kr/product/28465/73/co...,https://data4library.kr/bookV?seq=6360937
1,2,2,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001,03810,,813.7,문학 > 한국문학 > 소설,5186,https://image.aladin.co.kr/product/27692/63/co...,https://data4library.kr/bookV?seq=6065686
2,3,3,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,03810,,813.7,문학 > 한국문학 > 소설,5131,https://image.aladin.co.kr/product/26942/84/co...,https://data4library.kr/bookV?seq=5962217
3,4,4,달러구트 꿈 백화점 :이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,03810,,813.7,문학 > 한국문학 > 소설,5085,https://image.aladin.co.kr/product/24512/70/co...,https://data4library.kr/bookV?seq=5707051
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,03810,,813.7,문학 > 한국문학 > 소설,4730,https://image.aladin.co.kr/product/19359/16/co...,https://data4library.kr/bookV?seq=5430429


In [12]:
# 2. 특정 열을 선택하여 새 데이터프레임 생성 - 컬럼 인덱싱 이용
books_df.columns
books = books_df[['no','ranking','bookname','authors','publisher','publication_year','isbn13']]
books.head()

,no,ranking,bookname,authors,publisher,publication_year,isbn13
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156
1,2,2,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001
2,3,3,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188
3,4,4,달러구트 꿈 백화점 :이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018


In [ ]:
# method) 원하는 데이터프레임 행과 열 동시에 슬라이싱으로 선택하기 - df.loc[:,:]
books = books_df.loc[:, 'no':'isbn13']
books.head()

## 3. 추가 데이터 수집: 각 도서별 페이지수 찾기  
    *  HTML에서 데이터 추출하기 - BeautifulSoup
    1. 크롬 브라우저 개발자 도구로 HTML 태그 찾기
    2. 태그 위치 찾기 : soup.find('찾을 태그 이름',attrs={'태그 속성을 딕셔너리로 지정'} 
    3. soup.select()
    

### [문제1] BeautifulSoup으로 HTML에서 첫 번째 도서명의 링크 주소 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Search?query=9791189327156 # '물고기는 존재하지 않는다'의 ISBN

In [13]:
from bs4 import BeautifulSoup
import requests

In [16]:
isbn = 9791189327156
# 1. targt url
url = 'https://www.yes24.com/Product/Search?query={}'
url = url.format(isbn)
r = requests.get(url)
# 2. response message -> soup object
soup = BeautifulSoup(r.text, 'html.parser')
prd_link = soup.find('a', attrs={'class':'gd_name'})
print(prd_link)
print()
print(prd_link['href'])

<a class="gd_name" href="/Product/Goods/105526047" onclick="wiseLogV2('S', '101_005_003_001', ''); setGoodsClickExtraCodeHub('032', '9791189327156', '105526047', '0');">물고기는 존재하지 않는다</a>

/Product/Goods/105526047


### [문제2] BeautifulSoup으로 HTML에서 첫 번째 도서의 페이지수가 담긴 HTML의 위치 찾기
* 타겟 웹페이지 : https://www.yes24.com/Product/Goods/105526047
* soup.select()

In [21]:
# soup.select()
url = 'https://www.yes24.com/Product/Goods/105526047'
r= requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
# print(prd_detail) # list

if prd_detail is not None:
    # print(prd_detail[0].text.split('|'))
    page = prd_detail[0].text.split('|')[0]
    print(page)
else:
    print("The element you're trying to select doesn't exist.")


300쪽 


### [문제3]  20대가 선호하는 전체 도서의 쪽수 구하여 Series 만들기

In [22]:
#1. 사용자 정의 함수 : isbn을 전달받아 해당 도서의 페이지 수 반환하기
def get_page_cnt(isbn):
    # 1. Yes24 도서 검색 페이지 URL에 ISBN을 넣어 HTML 가져오기.
    url = 'https://www.yes24.com/Product/Search?query={}'
    r = requests.get(url.format(isbn))
    #2. soup 객체로 변환
    soup = BeautifulSoup(r.text, 'html.parser')   
    # 3.검색 결과에서 해당 도서를 선택하기.
    prd_info = soup.find('a', attrs={'class':'gd_name'})
    if prd_info == None:
        return ''
    # 4. 도서 상세 페이지를 가져오기.
    url = 'http://www.yes24.com'+prd_info['href']
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 5. 쪽수가 들어 있는 td를 찾기.
    prd_detail= soup.select('#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(2) > td')
    print(prd_detail)
    # 6. 페이지 수 추출하기
    result = prd_detail[0]
    page = result.text.split('|')[0]
    return page

# 2. 사용자 정의 함수 - 데이터프레임 데이터 도서의 쪽수를 한번에 반환하기
def get_multiple_page_cnt(row):
    isbn = row['isbn13']
    return get_page_cnt(isbn)

In [23]:
# 테스트 : 함수 호출
get_page_cnt(9791189327156)

[<td class="txt lastCol">300쪽 | 480g | 135*210*20mm</td>]


'300쪽 '

In [27]:
# 테스트 : 함수 호출
## 인기 도서 10권 추출 - 데이터프레임
top10_books = books.head(10) 
# top10_books
# 각 행의 반복 작업을 수행하기 위해 데이터프레임의 apply()메서드 적용(!!!!)
page_count = top10_books.apply(get_multiple_page_cnt, axis = 1)
print()
page_count
print(type(page_count))
# 전체 도서에 적용하면 ConnectionError 발생 가능
# page_count = books.apply(get_multiple_page_cnt, axis = 1)

[<td class="txt lastCol">300쪽 | 480g | 135*210*20mm</td>]
[<td class="txt lastCol">392쪽 | 420g | 130*187*20mm</td>]
[<td class="txt lastCol">268쪽 | 332g | 135*200*20mm</td>]
[<td class="txt lastCol">300쪽 | 358g | 134*200*20mm</td>]
[<td class="txt lastCol">
                                    양장
                                    <a class="bgYUI ico_comm ico_help" href="javascript:void(0);" onclick="$.yesPop('dPop_binding',this,{cock:true,mask:false,pWidth:640,ajaxURL : '/Product/Goods/Popup?id=dPop_bookBindingMethodInfo'});" title="도서 제본방식 안내">도서 제본방식 안내</a>
</td>]
[<td class="txt lastCol">340쪽 | 412g | 133*200*23mm</td>]
[<td class="txt lastCol">336쪽 | 360g | 128*188*16mm</td>]
[<td class="txt lastCol">344쪽 | 478g | 145*210*21mm</td>]
[<td class="txt lastCol">376쪽 | 434g | 130*198*25mm</td>]
[<td class="txt lastCol">165쪽 | 152g | 100*182*20mm</td>]

<class 'pandas.core.series.Series'>


In [28]:
# 쪽수 시리즈 생성
page_count = top10_books.apply(get_multiple_page_cnt, axis = 1)

type(page_count)
page_count.name = 'page_count'
print(page_count)

[<td class="txt lastCol">300쪽 | 480g | 135*210*20mm</td>]
[<td class="txt lastCol">392쪽 | 420g | 130*187*20mm</td>]
[<td class="txt lastCol">268쪽 | 332g | 135*200*20mm</td>]
[<td class="txt lastCol">300쪽 | 358g | 134*200*20mm</td>]
[<td class="txt lastCol">
                                    양장
                                    <a class="bgYUI ico_comm ico_help" href="javascript:void(0);" onclick="$.yesPop('dPop_binding',this,{cock:true,mask:false,pWidth:640,ajaxURL : '/Product/Goods/Popup?id=dPop_bookBindingMethodInfo'});" title="도서 제본방식 안내">도서 제본방식 안내</a>
</td>]
[<td class="txt lastCol">340쪽 | 412g | 133*200*23mm</td>]
[<td class="txt lastCol">336쪽 | 360g | 128*188*16mm</td>]
[<td class="txt lastCol">344쪽 | 478g | 145*210*21mm</td>]
[<td class="txt lastCol">376쪽 | 434g | 130*198*25mm</td>]
[<td class="txt lastCol">165쪽 | 152g | 100*182*20mm</td>]
0                                                300쪽 
1                                                392쪽 
2                         

## 4. 기존 데이터셋과 추가 데이터셋 합치기  
* pandas.merge(df, series, left_index= True, rught_index= True)    

### [문제]  top10_books와 쪽수 시리즈 합치기 

In [29]:
# 데이터프레임과 시리즈 합치기
top10_with_page_count = pd.merge(top10_books, page_count,
                                 left_index=True, right_index=True)
top10_with_page_count

,no,ranking,bookname,authors,publisher,publication_year,isbn13,page_count
0,1,1,"물고기는 존재하지 않는다 :상실, 사랑 그리고 숨어 있는 삶의 질서에 관한 이야기",룰루 밀러 지음 ;정지인 옮김,곰출판,2021,9791189327156,300쪽
1,2,2,지구 끝의 온실 :김초엽 장편소설,지은이: 김초엽,Giant Books(자이언트북스),2021,9791191824001,392쪽
2,3,3,불편한 편의점 :김호연 장편소설,지은이: 김호연,나무옆의자,2021,9791161571188,268쪽
3,4,4,달러구트 꿈 백화점 :이미예 장편소설,지은이: 이미예,팩토리나인,2020,9791165341909,300쪽
4,5,5,우리가 빛의 속도로 갈 수 없다면 :김초엽 소설,지은이: 김초엽,허블,2019,9791190090018,\r\n 양장\r\n...
5,6,6,"시선으로부터, :정세랑 장편소설",지은이: 정세랑,문학동네,2020,9788954672214,340쪽
6,7,7,홍학의 자리 :정해연 장편소설,지은이: 정해연,문학동네,2021,9788954681155,336쪽
7,8,8,밝은 밤 :최은영 장편소설,지은이: 최은영,문학동네,2021,9788954681179,344쪽
8,9,9,천 개의 파랑 :천선란 장편소설,지은이: 천선란,허블,2020,9791190090261,376쪽
9,10,10,"칵테일, 러브, 좀비",지은이: 조예은,안전가옥,2020,9791190174756,165쪽


## [연습] merge() 함수의 매개변수 이해하기

In [ ]:
df1 = pd.DataFrame({'col1': ['a','b','c'], 'col2': [1,2,3]})
df1

In [ ]:
df2 = pd.DataFrame({'col1': ['a','b','d'], 'col3': [10,20,30]})
df2

In [ ]:
#df1.merge(df2, on = 'col1')

In [ ]:
pd.merge(df1, df2, on='col1')


In [ ]:
pd.merge(df1, df2, how='left', on='col1')

In [ ]:
pd.merge(df1, df2, how='right', on='col1')

In [ ]:
pd.merge(df1, df2, how='outer', on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col1', right_on='col1')

In [ ]:
pd.merge(df1, df2, left_on='col2', right_index=True)